In [1]:
from Rings import Zomega, Ztau, Exact_U
from norm_equation import easy_factor, easy_solveable, solve_norm_equation
from exact_synthesize import exact_synthesize, exact_synthesize_weave
from Circuits import FTCircuit, Braid
from mpmath import mp, ceil, floor
import cmath
import math
import random
import numpy as np

To compile injection weaves, run all cells.

In [2]:
'''
Set computation depth
'''
mp.dps = 50

In [3]:
'''
Define global variables
'''

phi = (mp.sqrt(mp.mpf(5)) + 1)/2
tau = (mp.sqrt(mp.mpf(5)) - 1)/2

omega = Zomega(0,1,0,0)
T = Exact_U(Zomega(1,0,0,0), Zomega(0,0,0,0), 6)
F = Exact_U(Zomega(0,0,1,-1), Zomega(1,0,0,0), 0)

sigma_1 = Exact_U(omega**6, Zomega(0,0,0,0), 4)
sigma_2 = F * sigma_1 * F

def Rz(angle):
    return np.array([
        [cmath.exp(-0.5j*angle), 0],
        [0, cmath.exp(0.5j*angle)]
    ])

def Pauli_X():
    return np.array([
        [0, 1],
        [1, 0]
    ])

def F():
    return np.array([
        [tau, math.sqrt(tau)],
        [math.sqrt(tau), -1*tau]
    ])

In [4]:
'''
Implement random sample
'''
FibArray = [1, 1]
 
def Fib(n):
    if n<0:
        print("Incorrect input")
    elif n<= len(FibArray):
        return FibArray[n-1]
    else:
        temp_fib = Fib(n-1)+Fib(n-2)
        FibArray.append(temp_fib)
        return temp_fib    

def approx_real(x,n):
    p, q = Fib(n), Fib(n+1)
    u, v = (-1)**(n+1)*Fib(n), (-1)**n * Fib(n-1)
    c = round(x*q)
    a = c*v + p * round(c*u/q)
    b = c*u - q * round(c*u/q)

    return Ztau(a, b)

def random_sample(theta, epsilon, r):
    C = mp.sqrt(phi/(4*r))
    m = int(float(mp.nstr(mp.ceil(mp.log(C*epsilon*r, tau)), 1))) + 1
    N = int(float(mp.nstr(mp.ceil(phi**m), 1)))

    y_min = r*phi**m *(mp.sin(theta) - epsilon*(mp.sqrt(4-epsilon**2)*mp.cos(theta) + epsilon*mp.sin(theta))/2)
    y_max = r*phi**m *(mp.sin(theta) + epsilon*(mp.sqrt(4-epsilon**2)*mp.cos(theta) - epsilon*mp.sin(theta))/2)
    x_max = r*phi**m *((1 - epsilon**2/2)*mp.cos(theta) - epsilon*mp.sqrt(1 - epsilon**2/4)*mp.sin(theta))
    x_c = x_max - r*epsilon**2*phi**m/(4*mp.cos(theta))

    j = random.randint(1, N-1)
    y = y_min + j*(y_max - y_min)/N
    a_y = approx_real(y/mp.sqrt(2-tau), m)
    x = x_c - (a_y.value()*mp.sqrt(2-tau) - y_min)*mp.tan(theta)
    a_x = approx_real(x,m)

    return Zomega.fromTau(a_x) + Zomega(-1,2,-1,1)*Zomega.fromTau(a_y)

In [5]:
def compile_injection_weave(epsilon):
    weave = False
    while not weave:
        C = mp.sqrt(phi/4)
        m = int(float(mp.nstr(mp.ceil(mp.log(C*epsilon, tau)), 1))) + 1
        
        not_found, u, v = True, 0, 0

        while not_found:

            # Approximate identity
            u0 = random_sample(0, epsilon, 1)
            phi_t = Ztau(1, 1)
            xi = phi_t*(phi_t**(2*m) - Ztau.fromOmega(u0*u0.star()))
            factors = easy_factor(xi)
            if easy_solveable(factors):
                not_found = False
                u = Zomega.fromTau(Ztau(0,1))**m * u0
                v = Zomega.fromTau(Ztau(0,1))**m * solve_norm_equation(xi)
        
        #print("u, v = " + str(u) + ", " + str(v))
        #print("distance = " + str(mp.sqrt(1 - abs((u.value() + u.star().value()))/2)))   
        
        (Circuit, Remainder) = exact_synthesize_weave(sigma_1 * sigma_2 * Exact_U(u, v, 5))
        # print(Circuit)
        # print(Remainder)

        if len(Remainder.circuit) == 0:
            weave = True

    return Circuit

The box below compiles an injection weave to the desired precision. Works for precisions up to $10^{-6}$, though is finnicky in that some precisions only give certain phases. Weaves are printed until one has zero total phase.

In [6]:
'''
Test compile_injection_weave
works for precision = 0.0000004

Runtime is about 30s per try at this precision
'''
precision = 0.0000004

is_desired_phase = False
while not is_desired_phase:
    FT = compile_injection_weave(precision)
    weave = FT.toBraid()
    print(weave)
    if weave.phase == 0:
        is_desired_phase = True

exp(0.0πi)11112211222222111111112222222211111111221111111122222222111111112211222211221111221111222222221111111122222222111111222222221122222222112211222211111111222211111111222222111111112211111111222222221111111122221122222222112222222211111111222222221111111122111111112211221111
